In [ ]:
%%capture
!pip install llama-index==0.10.37 llama-index-embeddings-openai==0.1.9 qdrant-client==1.9.1 llama-index-vector-stores-qdrant==0.2.8 llama-index-llms-cohere==0.2.0

In [1]:
import os
from dotenv import load_dotenv
from getpass import getpass

import nest_asyncio

nest_asyncio.apply()
load_dotenv()

True

In [2]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

In [3]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")

In [4]:
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [5]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

# Query Pipelines

<img src="https://docs.llamaindex.ai/en/stable/_static/query/pipeline_rag_example.png">

Source: [LlamaIndex Docs](https://docs.llamaindex.ai/en/stable/module_guides/querying/pipeline/)

LlamaIndex offers a query API for chaining modules to manage data workflows easily. It revolves around the QueryPipeline, where you link various modules like LLMs, prompts, and retrievers in a sequence or DAG for end-to-end execution.

You can streamline workflows efficiently using QueryPipeline, reducing code complexity and enhancing readability. Additionally, a declarative interface ensures easy serialization of pipeline components for portability and deployment across systems in the future.

In [6]:
from llama_index.core.settings import Settings
from llama_index.llms.cohere import Cohere
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = Cohere(model="command-r-plus", api_key=CO_API_KEY)

Settings.embed_model = OpenAIEmbedding(model_name="text-embedding-3-small", api_key=OPENAI_API_KEY)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/conda/envs/lil_llama_index/lib/python3.10/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
from qdrant_client import QdrantClient
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

# Create a Qdrant client
client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_API_KEY,
)

# Create a Qdrant vector store
vector_store = QdrantVectorStore(
    client=client, 
    collection_name="it_can_be_done"
    )

# Create a vector store index
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=Settings.embed_model,
)

# A RAG Pipeline with PromptTemplate

I'm going to kick it off with a slightly complex workflow where the input is passes through two prompts before initiating retrieval.

1. Retrieve question about given topic.

2. Rephrase the context

Each prompt only takes in one input, so `QueryPipeline` will automatically chain LLM outputs into the prompt and then into the LLM.

You'll see how to define links more explicitly in the next section.

In [8]:
from llama_index.core.query_pipeline import QueryPipeline
from llama_index.core import PromptTemplate
from llama_index.core import PromptTemplate

# generate question regarding topic
prompt_str1 = "Retrieve context about the following topic: {topic}"
prompt_tmpl1 = PromptTemplate(prompt_str1)

prompt_str2 = """Syntesize the context provided into an answer using modern slang, while still quoting the sources.

Context:

{query_str}

Syntesized response:
"""

prompt_tmpl2 = PromptTemplate(prompt_str2)

retriever = index.as_retriever(similarity_top_k=5)

p = QueryPipeline(
    chain=[
        prompt_tmpl1, 
        retriever,
        prompt_tmpl2, 
        Settings.llm, 
        ], 
        verbose=True
)

In [9]:
response = p.run(topic="Working hard to achieve your goals even when you doubt yourself and your chances of success")

> Running module dfb467f2-feb4-455d-a00e-b5e94beb15cd with input: 
topic: Working hard to achieve your goals even when you doubt yourself and your chances of success

> Running module 4301c46b-40fa-4e59-b7d1-a6b516617158 with input: 
input: Retrieve context about the following topic: Working hard to achieve your goals even when you doubt yourself and your chances of success

> Running module 7e9c4087-ee0c-4a50-84d0-04124f8406f5 with input: 
query_str: [NodeWithScore(node=TextNode(id_='2a7697b2-b94f-4a28-96e4-f6ff49ca081d', embedding=None, metadata={'file_path': '../02_Fundamental_Concepts_in_LlamaIndex/data/pg10763.txt', 'file_name': 'pg10763.txt',...

> Running module a1367bdc-63de-41cc-af0d-5369d45504fa with input: 
messages: Syntesize the context provided into an answer using modern slang, while still quoting the sources.

Context:

["mountain with a\r\nmere pick and spade. We must assemble for the task great mechanical\r...



In [10]:
print(response)

assistant: So, like, there's this dude, Edgar A. Guest, and he's like, "Yo, if you wanna climb a mountain, you gotta bring more than just a pick and a spade, feel me? You gotta come correct with some major mechanical engineering, straight up." And I'm like, "Word?" But then he drops some knowledge about how our willpower is like a power tool – you bring a basic tool, you get a basic result. You bring a beast machine, you get beast mode results. 

So, then he's like, "How y'all approach your daily grind? You scared of the work, or you facing it head-on?" And he's right, ya feel? Like, if you're scared of your own potential, you ain't gonna get far. But if you believe in yourself and go hard, you can make that dream come true. He's like, "You can do anything you put your mind to, but you gotta put your mind to it first." And that's real talk. 

Then this other poet, Miriam Teichner, steps up and is like, "Yeah, we all wanna be heroes fighting dragons and stuff, but most days we just push

You can debug the pipeline by viewing intermediate inputs and outputs

In [11]:
output, intermediates  = p.run_with_intermediates(topic="Working hard to achieve your goals even when you doubt yourself and your chances of success")

> Running module dfb467f2-feb4-455d-a00e-b5e94beb15cd with input: 
topic: Working hard to achieve your goals even when you doubt yourself and your chances of success

> Running module 4301c46b-40fa-4e59-b7d1-a6b516617158 with input: 
input: Retrieve context about the following topic: Working hard to achieve your goals even when you doubt yourself and your chances of success

> Running module 7e9c4087-ee0c-4a50-84d0-04124f8406f5 with input: 
query_str: [NodeWithScore(node=TextNode(id_='2a7697b2-b94f-4a28-96e4-f6ff49ca081d', embedding=None, metadata={'file_path': '../02_Fundamental_Concepts_in_LlamaIndex/data/pg10763.txt', 'file_name': 'pg10763.txt',...

> Running module a1367bdc-63de-41cc-af0d-5369d45504fa with input: 
messages: Syntesize the context provided into an answer using modern slang, while still quoting the sources.

Context:

["mountain with a\r\nmere pick and spade. We must assemble for the task great mechanical\r...



In [12]:
output.__dict__

{'message': ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='So, like, there\'s this dude, Edgar A. Guest, and he\'s like, "Yo, if you wanna climb a mountain, you gotta bring more than just a pick and a spade, feel me? You gotta come correct with some major mechanical engineering, straight up." And I\'m like, "Word?" But then he drops some knowledge about how our willpower is like a power tool – you bring a basic tool, you get a basic result. You bring a beast machine, you get beast mode results.\n\nAnd then he\'s like, "How y\'all approach your daily grind? You scared of the work, or you facing it head-on? You feeling those dread vibes, or you out here manifesting that boss energy?" And I\'m just like, "Sheeesh, deep." \'Cause like, it\'s true, if you ain\'t feeling yourself, you ain\'t gonna get far. That\'s on period.\n\nThen this other poet, Miriam Teichner, slides in and starts spitting fire about how we all wanna be St. George, dragon-slaying and whatnot, but in re

In [13]:
intermediates

{'dfb467f2-feb4-455d-a00e-b5e94beb15cd': ComponentIntermediates(inputs={'topic': 'Working hard to achieve your goals even when you doubt yourself and your chances of success'}, outputs={'prompt': 'Retrieve context about the following topic: Working hard to achieve your goals even when you doubt yourself and your chances of success'}),
 '4301c46b-40fa-4e59-b7d1-a6b516617158': ComponentIntermediates(inputs={'input': 'Retrieve context about the following topic: Working hard to achieve your goals even when you doubt yourself and your chances of success'}, outputs={'output': [NodeWithScore(node=TextNode(id_='2a7697b2-b94f-4a28-96e4-f6ff49ca081d', embedding=None, metadata={'file_path': '../02_Fundamental_Concepts_in_LlamaIndex/data/pg10763.txt', 'file_name': 'pg10763.txt', 'file_type': 'text/plain', 'file_size': 405245, 'creation_date': '2025-02-04', 'last_modified_date': '2024-11-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date'

In [15]:
intermediates['dfb467f2-feb4-455d-a00e-b5e94beb15cd']

ComponentIntermediates(inputs={'topic': 'Working hard to achieve your goals even when you doubt yourself and your chances of success'}, outputs={'prompt': 'Retrieve context about the following topic: Working hard to achieve your goals even when you doubt yourself and your chances of success'})

### Another RAG Pipeline

Here we setup a RAG pipeline without the query rewriting step.

Here we need a way to link the input query to both the retriever and summarizer. 

We can do this by defining a special `InputComponent`, allowing us to link the inputs to multiple downstream modules.

In [17]:
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.query_pipeline import InputComponent
from llama_index.llms.openai import OpenAI

input = InputComponent()

retriever = index.as_retriever(similarity_top_k=5)

llm = OpenAI(model="gpt-4o")

tree_summarizer = TreeSummarize(llm=llm)

In [18]:
p = QueryPipeline(verbose=True, show_progress=True)

p.add_modules(
    {
        "input": input,
        "retriever": retriever,
        "tree_summarizer": tree_summarizer,
    }
)
p.add_link("input", "retriever")
p.add_link("input", "tree_summarizer", dest_key="query_str")
p.add_link("retriever", "tree_summarizer", dest_key="nodes")

In [19]:
response = p.run(input="Working hard to achieve your goals even when you doubt yourself and your chances of success")

> Running module input with input: 
input: Working hard to achieve your goals even when you doubt yourself and your chances of success

> Running module retriever with input: 
input: Working hard to achieve your goals even when you doubt yourself and your chances of success

> Running module tree_summarizer with input: 
query_str: Working hard to achieve your goals even when you doubt yourself and your chances of success
nodes: [NodeWithScore(node=TextNode(id_='2a7697b2-b94f-4a28-96e4-f6ff49ca081d', embedding=None, metadata={'file_path': '../02_Fundamental_Concepts_in_LlamaIndex/data/pg10763.txt', 'file_name': 'pg10763.txt',...



In [20]:
print(str(response))

Achieving your goals despite self-doubt and uncertain chances of success involves persistence, courage, and a strong will. It's about tackling each task with confidence and not letting fear or doubt hold you back. Success often requires quiet strength and unfaltering courage, even when the path is slow and painful. It's important to keep fighting against doubts and fears, as this struggle keeps your spirit strong and hopes alive. Progress may be slow, and victories might seem small, but maintaining a fighting spirit is crucial. Even when circumstances spoil your dreams or plans, continuing to strive and fight can eventually lead to success.


In [21]:
response.__dict__

{'response': "Achieving your goals despite self-doubt and uncertain chances of success involves persistence, courage, and a strong will. It's about tackling each task with confidence and not letting fear or doubt hold you back. Success often requires quiet strength and unfaltering courage, even when the path is slow and painful. It's important to keep fighting against doubts and fears, as this struggle keeps your spirit strong and hopes alive. Progress may be slow, and victories might seem small, but maintaining a fighting spirit is crucial. Even when circumstances spoil your dreams or plans, continuing to strive and fight can eventually lead to success.",
 'source_nodes': [NodeWithScore(node=TextNode(id_='2a7697b2-b94f-4a28-96e4-f6ff49ca081d', embedding=None, metadata={'file_path': '../02_Fundamental_Concepts_in_LlamaIndex/data/pg10763.txt', 'file_name': 'pg10763.txt', 'file_type': 'text/plain', 'file_size': 405245, 'creation_date': '2025-02-04', 'last_modified_date': '2024-11-05'}, e